# XAI CODE DEMO

## Explainable AI Specialization on Coursera

If you experience high latency while running this notebook, you can open it in Google Colab:

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/explainable-machine-learning/interpretable-ml/blob/main/rulefit_interpretability.ipynb)

# RuleFit

* RuleFit learns a sparse linear model with the original features AND also a number of new features that are decision rules
* New features that are decision rules capture interactions between the original features
* These features are generated from decision trees  trained to predict the outcome of interest

Steps:
1. Generate Rules
2. Create Sparse Linear Model

In this code demo, we will implement two versions of RuleFit. The first is an implementation via the imodels python library and the second is only using the scikit-learn package. Both implementations use a tree ensemble with gradient boosting and a Lasso linear model.

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

from imodels import RuleFitRegressor

#### Dataset

We will be using the Diabetes datastet for this demonstration: [LINK](https://www.geeksforgeeks.org/sklearn-diabetes-dataset/)

This dataset is from [Efron, et.al.](https://hastie.su.domains/Papers/LARS/LeastAngle_2002.pdf)

The features in this dataset are age, sex, bmi, blood pressure (bp), and six serum measurements (s1-s6).

In [2]:
# Load the diabetes dataset
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = diabetes.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## RuleFit Implementation with *imodels*
[imodels RuleFit algorithm](https://csinva.io/imodels/rule_set/rule_fit.html#imodels.rule_set.rule_fit.RuleFit)

From documentation:
Linear model of tree-based decision rules based on the rulefit algorithm from Friedman and Popescu.

The algorithm can be used for predicting an output vector y given an input matrix X. In the first step a tree ensemble is generated with gradient boosting. The trees are then used to form rules, where the paths to each node in each tree form one rule. A rule is a binary decision if an observation is in a given node, which is dependent on the input features that were used in the splits. The ensemble of rules together with the original input features are then being input in a L1-regularized linear model, also called Lasso, which estimates the effects of each rule on the output target but at the same time estimating many of those effects to zero.

### Initialize and train RuleFitRegressor

The below cell make take a few moments to run, since it is training our model.

In [3]:
# Initialize the RuleFitRegressor
model = RuleFitRegressor()

# Train the model
model.fit(X_train, y_train)

RuleFitRegressor()

### Evaluate Model

In [4]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')


Mean Squared Error: 2805.26


### Extract Rules

In [5]:
rule_df = model.visualize()
rule_df

,rule,coef
3,bp,9.24
20,bmi <= 0.07625 and s5 <= 0.022,-6.77
22,bmi <= 0.07894 and s5 <= 0.022,-0.08
18,bmi <= 0.07571 and s4 <= 0.05276 and s5 <= 0.02439,-8.76
12,bmi <= 0.00565 and s5 <= 0.0199,-14.83
14,bmi <= 0.00942 and s4 <= 0.03062,-2.43
16,bmi <= 0.00942 and s4 <= 0.03136,-3.73
19,bmi <= 0.00888 and s3 > -0.01947,-12.46
17,bmi <= -0.00674 and bp <= 0.02532,-2.37
13,s2 <= 0.01794 and s5 <= -0.00017,-16.20




---



## Implementing RuleFit without *imodels*

### Train a tree ensemble with gradient boosting

In [6]:
# Train the gradient boosting model
gb = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
gb.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

### Extract rules from the tree ensemble

Deduplicate rules

In [7]:
def extract_rules(tree, feature_names):
    rules = []
    tree_ = tree.tree_
    feature = tree_.feature
    threshold = tree_.threshold

    def traverse(node, rule):
        if tree_.feature[node] != -2:
            name = feature_names[feature[node]]
            threshold_value = threshold[node]
            left_rule = rule + [f"{name} <= {threshold_value}"]
            right_rule = rule + [f"{name} > {threshold_value}"]
            traverse(tree_.children_left[node], left_rule)
            traverse(tree_.children_right[node], right_rule)
        else:
            rules.append(rule)

    traverse(0, [])
    return rules

# Extract rules from all trees
rules = []
for estimator in gb.estimators_:
    for tree in estimator:
        rules.extend(extract_rules(tree, X.columns))

# Deduplicate rules
rules = list(map(list, {tuple(rule) for rule in rules}))


### Convert rules to feature matrix

In [8]:
def rule_to_feature_matrix(rules, X):
    feature_matrix = np.zeros((X.shape[0], len(rules)), dtype=int)
    for i, rule in enumerate(rules):
        rule_conditions = " & ".join(rule)
        feature_matrix[:, i] = X.eval(rule_conditions).astype(int)
    return feature_matrix

rule_features_train = rule_to_feature_matrix(rules, X_train)
rule_features_test = rule_to_feature_matrix(rules, X_test)

### Train Lasso model

Combine rule-based features with original features

In [9]:
# Combine original features and rule-based features
X_train_combined = np.hstack([X_train, rule_features_train])
X_test_combined = np.hstack([X_test, rule_features_test])

# Train the Lasso model
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_combined, y_train)

Lasso(alpha=0.1)

### Evaluate model

In [10]:
# Make predictions on the test set
y_pred = lasso.predict(X_test_combined)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

Mean Squared Error: 3710.12


### Extract Rules

In [11]:
# Get the coefficients and feature names (original features + rule-based features)
coefs = lasso.coef_
original_feature_names = X.columns.tolist()
rule_feature_names = [f"Rule {i+1}" for i in range(len(rules))]
all_feature_names = original_feature_names + rule_feature_names

# Sort coefficients and feature names by absolute coefficient value
coef_feature_pairs = sorted(zip(coefs, all_feature_names), key=lambda x: abs(x[0]), reverse=True)

# Print rules
for coef, feature_name in coef_feature_pairs:
    if coef != 0:
        if "Rule" in feature_name:
            rule_index = int(feature_name.split()[1]) - 1
            rule_description = " AND ".join(rules[rule_index])
            print(f"{feature_name}: Coefficient = {coef:.4f}, Rule = {rule_description}")
        else:
            print(f"{feature_name}: Coefficient = {coef:.4f}")


Rule 73: Coefficient = 80.5329, Rule = s6 > 0.04241442494094372 AND s5 > 0.01368608744814992 AND s5 <= 0.015194719657301903
Rule 319: Coefficient = 47.5309, Rule = bmi > 0.14899898320436478
Rule 581: Coefficient = 44.1219, Rule = bp <= 0.09818317741155624 AND s1 <= -0.024272182025015354 AND s1 > -0.028400040231645107
Rule 369: Coefficient = 43.9219, Rule = s5 <= -0.0010538420465309173 AND s2 > 0.017944753170013428 AND age <= -0.0581863634288311
Rule 276: Coefficient = -40.2026, Rule = s6 <= -0.003148751042317599 AND bp > 0.016708109062165022 AND s5 <= -0.03781251050531864
Rule 513: Coefficient = -32.8406, Rule = s4 <= 0.05441997013986111 AND s5 > 0.08501111716032028 AND bp <= 0.03220093855634332
Rule 394: Coefficient = -30.4126, Rule = age <= 0.007199329789727926 AND s2 > 0.016535584814846516 AND age > 0.003566791128832847
Rule 409: Coefficient = 29.8551, Rule = s5 <= 0.016671447083353996 AND bp <= 0.054579468443989754 AND bp > 0.03736521489918232
Rule 577: Coefficient = -28.4084, Rule